In [1]:
from nutri_checker.data.loader import load_nutri_data, COLUMNS_QUANTI, COLUMNS_QUALI
from nutri_checker.search.bm25 import NutriIndex
from nutri_checker.models.food import Aliment, Ingredient, Dish
import pandas as pd
from loguru import logger
import numpy as np

In [2]:
NUTRIDATA_PATH = "../data/table_Ciqual_2020_FR_20200707.xls"
NUTRI_MISSING_DATA_PATH = "../data/missing_values.json"

In [3]:
df = load_nutri_data(NUTRIDATA_PATH, NUTRI_MISSING_DATA_PATH)
index = NutriIndex(df)

In [10]:
meals = [
    {
        "name": "petit dej tartine miel",
        "ingredients": [
            ("beurre de cacahuètes", 25),
            ("miel", 25),
            ("pain intégral", 75),
        ],
    },
    {
        "name": "oeuf plat",
        "ingredients": [
            ("oeuf blanc", 60 * 65 / 100 * 2),
            ("oeuf jaune", 60 * 35 / 100 * 2),
            ("huile olive", 5),
        ],
    },
    {
        "name": "fondue de poireaux",
        "ingredients": [
            ("poireau", 100),
            ("oignon", 50),
            ("lait demi", 5),
            ("moutarde", 5),
            ("huile olive", 10),
        ],
    },
    {
        "name": "dahl",
        "ingredients": [
            ("oignon", 100 * 0.2),
            ("huile olive", 15 * 0.2),
            ("lentille corail", 250 * 0.2),
            ("lait coco", 200 * 0.2),
            ("sauce tomate", 160 * 0.2),
            ("curry", 8 * 0.2),
            ("beurre cacahuètes", 20 * 0.2),
            ("ail cru", 9 * 0.2),
            ("carottes cuites", 145 * 0.2),
            ("navet cuites", 250 * 0.2),
        ],
    },
]

In [12]:
dishes = []
for meal in meals:
    dish = Dish(
        name=meal["name"],
        ingredients=[
            Ingredient(Aliment.from_name_in_nutridata(name, index), quantity_g=quantity)
            for name, quantity in meal["ingredients"]
        ],
    )

2025-02-28 09:53:45.463 | WARNING  | nutri_checker.models.food:__post_init__:67 - Aliment miel is not raw food
2025-02-28 09:53:45.465 | WARNING  | nutri_checker.models.food:__post_init__:67 - Aliment huile d'olive vierge extra is not raw food
2025-02-28 09:53:45.470 | WARNING  | nutri_checker.models.food:__post_init__:67 - Aliment lait demi-écrémé, pasteurisé is not raw food
2025-02-28 09:53:45.472 | WARNING  | nutri_checker.models.food:__post_init__:67 - Aliment huile d'olive vierge extra is not raw food
2025-02-28 09:53:45.474 | WARNING  | nutri_checker.models.food:__post_init__:67 - Aliment huile d'olive vierge extra is not raw food


In [5]:
d = Dish(
    "test",
    ingredients=[
        Ingredient(Aliment.from_name_in_nutridata("huile olive", index), 30),
        Ingredient(Aliment.from_name_in_nutridata("viande poulet cru", index), 200),
    ],
)

2025-03-01 19:54:27.875 | WARNING  | nutri_checker.models.food:__post_init__:67 - Aliment huile d'olive vierge extra is not raw food


In [10]:
index.retrieve("huile olive")

,grp_name_fr,ssgrp_name_fr,ssssgrp_name_fr,name_fr,grp_code,ssgrp_code,ssssgrp_code,code,energy_kcal,water_g,...,vit_b9_ug,vit_b12_ug,sodium_mg,phosphore_mg,selenium_ug,zinc_mg,chlorure_mg,cu_mg,mn_mg,raw_food
2895,matières grasses,huiles et graisses végétales,-,huile d'olive vierge extra,9,902,0,17270,9.0,0.001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False


In [9]:
d.get_nutri()

,grp_name_fr,ssgrp_name_fr,ssssgrp_name_fr,name_fr,grp_code,ssgrp_code,ssssgrp_code,code,energy_kcal,water_g,...,vit_b6_mg,vit_b9_ug,vit_b12_ug,sodium_mg,phosphore_mg,selenium_ug,zinc_mg,chlorure_mg,cu_mg,mn_mg
0,matières grasses,huiles et graisses végétales,-,huile d'olive vierge extra,9,902,0,17270,270.0,0.03,...,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.000
1,"viandes, œufs, poissons et assimilés",viandes crues,poulet,"poulet, viande et peau, cru",4,402,40203,36016,346.0,136.20,...,1.08,20.8,2.58,124.6,348.0,24.8,2.6,NaN,0.28,0.052


In [ ]:
class Menu:
    def __init__(self, name, dishes: List[Dish]):
        self.name = name
        self.dishes = dishes

    def __str__(self):
        return f"{self.name} ({len(self.dishes)} dishes)"

    def __repr__(self):
        return f"{self.name} ({len(self.dishes)} dishes)"

    def __len__(self):
        return len(self.dishes)

    def __iter__(self):
        return iter(self.dishes)

    def __getitem__(self, idx):
        return self.dishes[idx]

    def __setitem__(self, idx, value):
        self.dishes[idx] = value

    def __delitem__(self, idx):
        del self.dishes[idx]

    def __contains__(self, item):
        return item in self.dishes

    def __add__(self, other):
        return Menu(self.name + " & " + other.name, self.dishes + other.dishes)

    def __iadd__(self, other):
        self.dishes += other.dishes
        return self

    def get_portion(self, portion_g):
        return Menu(self.name, [dish.get_portion(portion_g) for dish in self.dishes])

    def get_nutri(self):
        return pd.DataFrame(data=[dish.get_nutri() for dish in self.dishes]).sum()

In [14]:
a = Dish(
    "a",
    [
        Ingredient(Aliment.from_row(df.iloc[0]), 100),
        Ingredient(Aliment.from_row(df.iloc[10]), 100),
    ],
)

In [15]:
a.get_nutri()

grp_name_fr           fruits, légumes, légumineuses et oléagineuxfru...
ssgrp_name_fr                                            légumeslégumes
ssssgrp_name_fr                                légumes cruslégumes crus
name_fr                           avocat, pulpe, crucéleri branche, cru
grp_code                                                              4
ssgrp_code                                                          402
ssssgrp_code                                                      40202
code                                                              33027
energy_kcal                                                       222.6
water_g                                                           163.7
prot_g                                                             2.19
glucid_g                                                           3.24
lipid_g                                                            20.6
sugar_g                                                         